In [1]:
!pip install datasets
!pip install tokenizers
!pip install torchmetrics
!pip install lightning
!pip install pytorch_lightning


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.4/763.4 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.7/596.7 kB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import lightning.pytorch as pl
from lightning.pytorch.loggers import TensorBoardLogger,CSVLogger
import torch
import torch.nn as nn
import warnings
warnings.filterwarnings("ignore")
import logging
from torch.optim.lr_scheduler import OneCycleLR
import seaborn as sn
import pandas as pd
import torchmetrics

from config import get_config
from train import get_ds
from model import build_transformer

warnings.filterwarnings("ignore")

logger = logging.getLogger("Transformer")
logger.setLevel(level=logging.INFO)
fileHandler = logging.FileHandler(filename='prediction.log')
fileHandler.setLevel(level=logging.INFO)
logger.addHandler(fileHandler)

config = get_config()
config['preload'] = None
config['num_epochs'] = 35
config['batch_size'] = 16

In [2]:
class LighteningTransformer_Faster(pl.LightningModule):
    def __init__(self,config,tokenizer_src,tokenizer_tgt):
        super().__init__()

        vocab_src_len = tokenizer_src.get_vocab_size()
        vocab_tgt_len = tokenizer_tgt.get_vocab_size()
        self.model    = build_transformer(vocab_src_len, vocab_tgt_len, config['seq_len'], config['seq_len'], d_model=config['d_model'])
        self.loss_fn  = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id('[PAD]'), label_smoothing=0.1)
        self.num_epochs = config['num_epochs']
        self.lr = config['lr']
        self.tokenizer_src = tokenizer_src
        self.tokenizer_tgt = tokenizer_tgt
        self.seq_len = config['seq_len']

        self.training_loss = []

        self.expected = []
        self.predicted = []

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):

        encoder_input = batch['encoder_input']
        decoder_input = batch['decoder_input']
        encoder_mask  = batch['encoder_mask']
        decoder_mask  = batch['decoder_mask']

        # Run the tensors through the encoder, decoder and the projection layer
        encoder_output = self.model.encode(encoder_input, encoder_mask)
        decoder_output = self.model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask)
        proj_output = self.model.project(decoder_output)

        label = batch['label']

        loss = self.loss_fn(proj_output.view(-1, self.tokenizer_tgt.get_vocab_size()), label.view(-1))
        self.training_loss.append(loss)

        self.log('training_loss', loss, prog_bar=True, on_step=False, on_epoch=True,logger=True)
        return loss

    def on_train_epoch_end(self):
        epoch = self.current_epoch
        print("----------------------------------------------------------------------")
        mean_train_loss = sum(self.training_loss) / len(self.training_loss)
        print(f'Epoch Number: {epoch}')
        # Print the final training loss
        print("Training Loss   : ", round(mean_train_loss.item(), 5))
        print("-------------------------------------------------------------------------------------------------")
        self.training_loss = []

    def casual_mask(self, size):
        mask = torch.triu(torch.ones((1, size, size)), diagonal=1).type(torch.int)
        return(mask == 0)

    def greedy_decode(self,source, source_mask):

        sos_idx = self.tokenizer_tgt.token_to_id('[SOS]')
        eos_idx = self.tokenizer_tgt.token_to_id('[EOS]')

        encoder_output = self.model.encode(source, source_mask)
        decoder_input = torch.empty(1,1).fill_(sos_idx).type_as(source)

        while True:
            if decoder_input.size(1) == self.seq_len:
              break

            decoder_mask = self.casual_mask(decoder_input.size(1)).type_as(source_mask)

            out = self.model.decode(encoder_output, source_mask, decoder_input, decoder_mask)

            prob = self.model.project(out[:,-1])
            _,next_word = torch.max(prob, dim=1)

            decoder_input = torch.cat(
                [decoder_input, torch.empty(1,1).type_as(source).fill_(next_word.item())], dim=1
            )

            if next_word == eos_idx:
              break

        return decoder_input.squeeze(0)

    def validation_step(self, batch, batch_idx):
        encoder_input = batch["encoder_input"]
        encoder_mask  = batch["encoder_mask"]
        model_out = self.greedy_decode(encoder_input, encoder_mask)
        source_text = batch["src_text"][0]
        target_text = batch["tgt_text"][0]
        model_out_text = self.tokenizer_tgt.decode(model_out.detach().cpu().numpy())

        source_text = self.tokenizer_src.decode(batch["encoder_input"][0].detach().cpu().numpy())
        print("------------------------------------------------------------------------------------")
        print(f"{f'SOURCE: ':>12}{source_text}")
        print(f"{f'TARGET: ':>12}{target_text}")
        print(f"{f'PREDICTED: ':>12}{model_out_text}")
        print("----------------------------------------------------------------------")
        self.expected.append(target_text)
        self.predicted.append(model_out_text)


    def on_validation_epoch_end(self):
        metric = torchmetrics.CharErrorRate()
        cer = metric(self.predicted, self.expected)
        print(f"Validation CER  : {cer}")
        self.log('validation_cer', cer, prog_bar=True, on_epoch=True, logger=True)


        # Compute the word error rate
        metric = torchmetrics.WordErrorRate()
        wer = metric(self.predicted, self.expected)
        self.log('validation_wer', wer, prog_bar=True, on_epoch=True, logger=True)
        print(f"Validation WER  : {wer}")
        print("----------------------------------------------------------------------")

        self.expected.clear()
        self.predicted.clear()

    def optimizer_zero_grad(self, epoch, batch_idx, optimizer):
        optimizer.zero_grad(set_to_none=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr = self.lr, eps = 1e-9)
        scheduler = OneCycleLR(
                        optimizer,
                        max_lr= 10**-3,
                        pct_start = 1/10,
                        epochs=self.trainer.max_epochs,
                        steps_per_epoch=len(train_dataloader),
                        div_factor=10,
                        three_phase=True,
                        final_div_factor=10,
                        anneal_strategy='linear'
                    )
        return {
             "optimizer": optimizer,
             "lr_scheduler": {"scheduler": scheduler, "interval": "step"},
             }

In [ ]:
train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)

trainer = pl.Trainer(limit_val_batches=2,
                     precision=16,
                     check_val_every_n_epoch=1,
                     max_epochs = config['num_epochs'],
                     devices='auto',
                     accelerator='auto',
                     strategy='auto',
                     logger=[TensorBoardLogger("logs/", name="logging"),CSVLogger(save_dir="logs/")],
    )

model = LighteningTransformer_Faster(config,tokenizer_src,tokenizer_tgt)
trainer.fit(model, train_dataloader ,val_dataloader)


Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

Length of filtered dataset : 68604
Train DS Size : 61743
  Val DS Size : 6861
Max length of source sentence: 45
Max length of target sentence: 48
Source Vocab Size : 30000
Target Vocab Size : 30000


INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loading `train_dataloader` to estimate number of stepping 

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

Epoch 14 :
----------------------------------------------------------------------
SOURCE    := Leaning over, with wide eyes, Étienne looked at him.
EXPECTED  := Et, penché, l'oeil élargi, Étienne le regardait.
PREDICTED := - le de la large , Étienne le regarda .
----------------------------------------------------------------------
Validation CER  : 0.3958333432674408
Validation WER  : 1.1428571939468384
Validation BLEU : 0.0
Final Training Loss   :  1.82805


Validation: 0it [00:00, ?it/s]

Epoch 15 :
----------------------------------------------------------------------
SOURCE    := Leaning over, with wide eyes, Étienne looked at him.
EXPECTED  := Et, penché, l'oeil élargi, Étienne le regardait.
PREDICTED := - le de ses yeux , Étienne le regarda .
----------------------------------------------------------------------
Validation CER  : 0.4583333432674408
Validation WER  : 1.1428571939468384
Validation BLEU : 0.0
Final Training Loss   :  1.79095


Validation: 0it [00:00, ?it/s]

Epoch 16 :
----------------------------------------------------------------------
SOURCE    := Leaning over, with wide eyes, Étienne looked at him.
EXPECTED  := Et, penché, l'oeil élargi, Étienne le regardait.
PREDICTED := - moi de grands yeux , Étienne le regarda .
----------------------------------------------------------------------
Validation CER  : 0.5
Validation WER  : 1.1428571939468384
Validation BLEU : 0.0
Final Training Loss   :  1.77434
